# An input file is generated for ASAXS data correction
\
Read the headers of all .edf files from all directories in the destination directory.
Headers sould containing data for the SAXS data reduction \
An output file is created which contains information from all files in the directory. \
\
The entries to read from the files are defined in >>>deviceList<<< \
Entries to latere set by hand are defined in >>>deviceByHandList<<< \
\
The following information must be entered concerning the files: \
The file directory \
The file data type like .edf \

The structure of entries in the header (seperator, digits to skip after entry \

## Libraries

In [2]:
import pandas as pd
import os
from scantree import scantree, RecursionFilter
import fabio

## Do the job
First the nessesary information

In [97]:
# give the metadada directory
data_dir = 'C:\\Users\\eikeg\\Documents\\Science\\qe05_ESRF\\3th-reaction'

# give the target file tyle:
target_file_type = '.edf'
searchTree = True #Type true to seach in entire file tree, type false to seach only the particular directory

# give the output file name
outputFileName = data_dir.split(os.sep)[-1] + '_ASCII_out.txt'

# give perimental X-ray energy as string
expEnergy = '11993' #eV

# Concerning the header
# Number of header line
header_lines = 375
# Structure of information in header
seperatorBeforeInfo = '\t'
seperatorAfterInfo = '\t = '
digitsAfterIdentifier = 12 # this is the length of the entry after the identifier you are searching for

#### BESSY II - PTB SAXS List ###
# List of experimental divices/information nessesary for the data reduction
#deviceList = ['Time','VacSampleX', 'VacSampleY','Energy','Pilatus_Tiff','Pilatus_Trigger','Pilatus_filename','PilatusAcqTime','Keysight1','Keysight2','Keysight3','Keysight4','Keysight1:StandardDeviation','Keysight2:StandardDeviation','Keysight3:StandardDeviation','Keysight4:StandardDeviation','PilatusPos']
### ESRF BM26 SAXS List ###
deviceList = ['time','sampleX', 'sampleY','FrameTime','Monitor','Photo']
# List of values to set by hand
deviceByHandList = ['SampleName','SampleThickness','Empty_Index','Background_Index','Reference_Index','MaskFile']
# List of values to change by hand if nessesary
deviceStandardNameList = ['Energy','BackGround_SubtractionFactor','x_Center','y_Center','SampleToDet_Distance']
deviceStandardValueList = [expEnergy,'1.0','497.663','25.230','2.54696']

Now the iteration an export

In [104]:
fileList = list()
if searchTree == True:
    tree = scantree(data_dir, RecursionFilter(match=['*'+target_file_type]))
    for path in tree.filepaths():
        fileList.append(path.real)
elif searchTree == False:
    for entry in os.scandir(data_dir):
        if entry.path.endswith(target_file_type) and entry.is_file():
            fileList.append(entry.path)
else:
    print("Error: Specify if to search target file in entire file Tree")

len(fileList)

367

In [107]:
# make an path iterator from the found files
fileIterator = iter(fileList)
metaFrame=pd.DataFrame()
for element in fileIterator:
    #print(element)
    deviceFrame = pd.DataFrame({'fileName':element.split(os.sep)[-1]}, index=[0]) 
    deviceFrame['filePath']=element
    deviceIterator = iter(deviceList)
    
    image_object = fabio.open(element)
    image_header = image_object.header
    
    for device in deviceIterator:
        deviceFrame[device]=image_header[device]
        #print(device,'=',image_header[device])
        
    metaFrame = metaFrame.append(deviceFrame)

metaFrame = metaFrame.reset_index(drop=True)
for device in deviceByHandList:
    metaFrame[device] = 'NaN'
for device in deviceStandardNameList:
    metaFrame[device] = deviceStandardValueList[deviceStandardNameList.index(device)]
    
out_path = data_dir + os.sep + outputFileName
metaFrame.to_csv(out_path, sep='\t')

In [106]:
metaFrame

,fileName,filePath,time,sampleX,sampleY,FrameTime,Monitor,Photo,SampleName,SampleThickness,Empty_Index,Background_Index,Reference_Index,MaskFile,Energy,BackGround_SubtractionFactor,x_Center,y_Center,SampleToDet_Distance
0,3th-reaction_UV-light_SAXS_0000.edf,C:\Users\eikeg\Documents\Science\qe05_ESRF\3th...,Sat May 6 22:04:14 2017,0,-14.9825,14.000000,3623316,101739,NaN,NaN,NaN,NaN,NaN,NaN,11993,1.0,497.663,25.230,2.54696
1,3th-reaction_UV-light_SAXS_0001.edf,C:\Users\eikeg\Documents\Science\qe05_ESRF\3th...,Sat May 6 22:04:29 2017,0,-14.9825,29.000000,3620284,104048,NaN,NaN,NaN,NaN,NaN,NaN,11993,1.0,497.663,25.230,2.54696
2,3th-reaction_UV-light_SAXS_0002.edf,C:\Users\eikeg\Documents\Science\qe05_ESRF\3th...,Sat May 6 22:04:44 2017,0,-14.9825,44.000000,3599123,103331,NaN,NaN,NaN,NaN,NaN,NaN,11993,1.0,497.663,25.230,2.54696
3,3th-reaction_UV-light_SAXS_0003.edf,C:\Users\eikeg\Documents\Science\qe05_ESRF\3th...,Sat May 6 22:04:59 2017,0,-14.9825,59.000000,3616451,103892,NaN,NaN,NaN,NaN,NaN,NaN,11993,1.0,497.663,25.230,2.54696
4,3th-reaction_UV-light_SAXS_0004.edf,C:\Users\eikeg\Documents\Science\qe05_ESRF\3th...,Sat May 6 22:05:14 2017,0,-14.9825,74.000000,3625962,104218,NaN,NaN,NaN,NaN,NaN,NaN,11993,1.0,497.663,25.230,2.54696
5,3th-reaction_UV-light_SAXS_0005.edf,C:\Users\eikeg\Documents\Science\qe05_ESRF\3th...,Sat May 6 22:05:29 2017,0,-14.9825,89.000000,3618521,103954,NaN,NaN,NaN,NaN,NaN,NaN,11993,1.0,497.663,25.230,2.54696
6,3th-reaction_UV-light_SAXS_0006.edf,C:\Users\eikeg\Documents\Science\qe05_ESRF\3th...,Sat May 6 22:05:44 2017,0,-14.9825,104.000000,3622802,104114,NaN,NaN,NaN,NaN,NaN,NaN,11993,1.0,497.663,25.230,2.54696
7,3th-reaction_UV-light_SAXS_0007.edf,C:\Users\eikeg\Documents\Science\qe05_ESRF\3th...,Sat May 6 22:05:59 2017,0,-14.9825,119.000000,3607710,103601,NaN,NaN,NaN,NaN,NaN,NaN,11993,1.0,497.663,25.230,2.54696
8,3th-reaction_UV-light_SAXS_0008.edf,C:\Users\eikeg\Documents\Science\qe05_ESRF\3th...,Sat May 6 22:06:14 2017,0,-14.9825,134.000000,3618485,103971,NaN,NaN,NaN,NaN,NaN,NaN,11993,1.0,497.663,25.230,2.54696
9,3th-reaction_UV-light_SAXS_0009.edf,C:\Users\eikeg\Documents\Science\qe05_ESRF\3th...,Sat May 6 22:06:29 2017,0,-14.9825,149.000000,3614909,103850,NaN,NaN,NaN,NaN,NaN,NaN,11993,1.0,497.663,25.230,2.54696


In [69]:
image_header[deviceList[0]]

'Fri May  5 16:06:12 2017'

In [83]:
device = 'file'
pd.DataFrame({device:'test'}, index=[0])

,file
0,test
